In [1]:
import json

import tensorflow as tf
import numpy as np
import os
import data_helpers
from data_loader import DataLoader
from word_data_processor import WordDataProcessor
import csv

In [2]:
# Parameters
# ==================================================

# Eval Parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
# 1496390706 filter_sizes:3,4,5 num_epochs:10
# 1496630751 2,3,4 num_epochs: 10
# 1496997900 2,3,4 num_epochs:2
# 1497262314 2,3,4 num_epochs : 30
# 1497442275 2,3,4 num_epochs : 10  l2_reg_lambda : 0.5
tf.flags.DEFINE_string("checkpoint_dir", "/root/notebooks/dawoon/runs/1497442275/checkpoints/", "Checkpoint directory from training run")
tf.flags.DEFINE_boolean("eval_train", True, "Evaluate on all training data")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

data_loader = DataLoader(tf.flags, WordDataProcessor())
data_loader.define_flags()

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


Parameters:
ALLOW_SOFT_PLACEMENT=True
BAG_OF_WORD_DATA_FILE=bow.out
BATCH_SIZE=64
CHECKPOINT_DIR=/root/notebooks/dawoon/runs/1497442275/checkpoints/
DEV_DATA_FILE=test-sms_word_vector.csv
EVAL_TRAIN=True
LOG_DEVICE_PLACEMENT=False
TRAIN_DATA_FILE=data-sms_word_vector.csv



In [3]:
if FLAGS.eval_train:
    #x_test, y_test = data_loader.load_data_and_labels()
    x_test = np.array([[2,45,1,3,1,43584,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                       ,[1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                       ,[2,1022,1,3,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                       ,[2,10,12,3,1,1,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                        ,[2,1,3,1,4197,17,34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                      ,[2,3698,1,22396,13875,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                      ,[2,37899,1,42,66993,4197,1,3,43,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                      ,[2,1,1,1,1,34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])
    y_test = [0,0,1,0,1,1,1,1]
    #print("x_test.shape", x_test.shape)
    #print("x_test[0]", x_test[0])
else:
    x_test, y_test = data_loader.load_dev_data_and_labels()
    print("x_test.shape", x_test.shape)
    print("x_test[0]", x_test[0])
    print("y_test.shape", y_test.shape)
    print("y_test[0]", y_test[0])
    y_test = np.argmax(y_test, axis=1)
    print("y_test.shape", y_test.shape)
    print("y_test[0]", y_test[0])
    
    
    

In [4]:
# checkpoint_dir이 없다면 가장 최근 dir 추출하여 셋팅
if FLAGS.checkpoint_dir == "":
    all_subdirs = ["./runs/" + d for d in os.listdir('./runs/.') if os.path.isdir("./runs/" + d)]
    latest_subdir = max(all_subdirs, key=os.path.getmtime)
    FLAGS.checkpoint_dir = latest_subdir + "/checkpoints/"

In [5]:
# Map data into vocabulary
# restore_vocab_processor는 이미 전처리 되어서 생략

print("\nEvaluating...\n")
checkpoint_file = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)
print ("\ncheckpoint_file", checkpoint_file)
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)

        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        # input_y = graph.get_operation_by_name("input_y").outputs[0]
        #operations = graph.get_operations()
        #print("operations", operations)
        
        # sms_cnn.py에 오타가 있었음 dropout_keep_prob -> dropout_keep_porb
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0] 
        #dropout_keep_prob = graph.get_operation_by_name("dropout_keep_porb").outputs[0]
        

        # Tensors we want to evaluate
        #embedding/output/predictions 또 인덴트 오타가 있었음ㅠ 
        #predictions = graph.get_operation_by_name("embedding/output/predictions").outputs[0]
        predictions = graph.get_operation_by_name("output/predictions").outputs[0]
        
        
        # Generate batches for one epoch
        batches = data_helpers.batch_iter(list(x_test), FLAGS.batch_size, 1, shuffle=False)

        # Collect the predictions here
        all_predictions = []

        for x_test_batch in batches:
            batch_predictions = sess.run(predictions, {input_x: x_test_batch, dropout_keep_prob: 1.0})
            all_predictions = np.concatenate([all_predictions, batch_predictions])



Evaluating...

('\ncheckpoint_file', u'/root/notebooks/dawoon/runs/1497442275/checkpoints/model-28000')


In [6]:
# Print accuracy if y_test is defined
if y_test is not None:
    correct_predictions = float(sum(all_predictions == y_test ))
    correct_predictions_of1 = float(sum(all_predictions == y_test))
    print("Total number of test examples: {}".format(len(y_test)))
    print("Accuracy: {:g}".format(correct_predictions/float(len(y_test))))

    
    if FLAGS.eval_train:
        for idx, x in enumerate(x_test): 
            if y_test[idx] != all_predictions[idx]:
                for x_word in x:
                    if x_word != 0:
                        print (data_loader.get_word_by_id(str(x_word)))
                print("")
        print("y_test", y_test)
        print("all_predictions", all_predictions)
# Save the evaluation to a csv
#class_predictions = data_loader.class_labels(all_predictions.astype(int))
#predictions_human_readable = np.column_stack((np.array(x_raw), class_predictions))
#out_path = os.path.join(FLAGS.checkpoint_dir, "../../../", "prediction.csv")
#print("Saving evaluation to {0}".format(out_path))
#with open(out_path, 'w') as f:
    #csv.writer(f).writerows(predictions_human_readable)

Total number of test examples: 8
Accuracy: 0.75
[Web발신]
KB국민카드
UNKNOWN
DATETIME
UNKNOWN
오류

[Web발신]
UNKNOWN
UNKNOWN
UNKNOWN
UNKNOWN
누적_MONEY_WON

('y_test', [0, 0, 1, 0, 1, 1, 1, 1])
('all_predictions', array([ 1.,  0.,  1.,  0.,  1.,  1.,  1.,  0.]))
